### Задача: классификация твитов по тональности

*sentiment analysis*

У нас есть данные постов в твиттере, про каждый из которых каждый указано, как он эмоционально окрашен: положительно или отрицательно.

**Задача**: построить модель, которая по тексту поста предсказывает его эмоциональную окраску.


Скачиваем данные: положительные, отрицательные.

*Спасибо авторам тетрадки с курса [Анализ текстовых данных](https://openedu.ru/course/hse/TEXT/)*

Кстати, суперкурс (но там без объяснений математики для неподготовленных)

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1VpeYPhqxfGF6ZTKOEZ56Ew4EU2OzXbSA' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1VpeYPhqxfGF6ZTKOEZ56Ew4EU2OzXbSA" -O negative.csv && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UFv55ApwTvvBpkRCLucD6SHlMHuDIVR-' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1UFv55ApwTvvBpkRCLucD6SHlMHuDIVR-" -O positive.csv && rm -rf /tmp/cookies.txt

--2024-09-05 10:22:46--  https://docs.google.com/uc?export=download&confirm=&id=1VpeYPhqxfGF6ZTKOEZ56Ew4EU2OzXbSA
Resolving docs.google.com (docs.google.com)... 173.194.195.100, 173.194.195.102, 173.194.195.138, ...
Connecting to docs.google.com (docs.google.com)|173.194.195.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1VpeYPhqxfGF6ZTKOEZ56Ew4EU2OzXbSA&export=download [following]
--2024-09-05 10:22:46--  https://drive.usercontent.google.com/download?id=1VpeYPhqxfGF6ZTKOEZ56Ew4EU2OzXbSA&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.184.132, 2607:f8b0:4001:c66::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.184.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24450101 (23M) [application/octet-stream]
Saving to: ‘negative.csv’

negative.csv        100%[===================>]  23.32M  

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

pd.set_option('max_colwidth', 800)

In [ ]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive.head()

,text
0,"@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)"
1,"Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D"
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за тебя!!!
3,"RT @digger2912: ""Кто то в углу сидит и погибает от голода, а мы ещё 2 порции взяли, хотя уже и так жрать не хотим"" :DD http://t.co/GqG6iuE2…"
4,"@irina_dyshkant Вот что значит страшилка :D\nНо блин,посмотрев все части,у тебя создастся ощущение,что авторы курили что-то :D"


In [ ]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = pd.concat([positive, negative])

In [ ]:
df.sample(5)

,text,label
51861,"@thebestfun1d блин, давай ты вопросы задавать будешь? Я так не могу х)",positive
73629,"Чтобы вещь нашлась, нужно перерыть весь дом и кого-нибудь заподозрить в ее пропаже..)",positive
64612,RT @domanep: теперь тоже планирую ходить по магазинам в одеяле) http://t.co/SYqPIg2Eb9,positive
70844,@Yuri_Romanov Ну вот как -то не верится!!!((( Если только также как сейчас?,negative
70471,"@agalitsky солнечно! :) привезите в москву кусочек хорошей погоды, хотя сегодня и здесь солнце весь день!",positive


In [ ]:
df.shape

(226834, 2)

Предобработаем текст: переведём в список лемм.

In [ ]:
! pip install pymorphy2

In [ ]:
import re
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
#LRU (least recently used) кеш - кеш с конечным размером,
# где часто используемые записи вытесняют прочие.
from nltk.corpus import stopwords

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
m = MorphAnalyzer()
regex = re.compile("[А-ЯЁа-яёA-z]+")  # для буквенных сочетаний (=слов)

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

In [ ]:
@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]

mystopwords = stopwords.words('russian')
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 2]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)

    return ' '.join(remove_stopwords(lemmas))

In [ ]:
from tqdm import tqdm

In [ ]:
lemmas = list(tqdm(map(clean_text, df['text'])))  # это занимает 8 минут

df['lemmas'] = lemmas
df.sample(5)

226834it [08:11, 461.15it/s]


,text,label,lemmas
17209,"RT @Atvechayu: У каждого есть друг, который: \n1) Ест как свинья и не толстеет; \n2) Постоянно бухает; \n3) Постоянно ругается матом; \n4) Ржач…",positive,atvechayu каждый друг который свинья толстеть постоянно бухать постоянно ругаться ржач
74329,Переделала уже много дел.теперь можно немного поспать.есть ещё пару часиков...)главное не проспать:-),positive,переделать дело немного поспать пара часик главное проспать
71785,RT @Tbl_HA_MAPCE: @chlenschmidta не за что бабуська)))))0,positive,tbl_ha_mapce chlenschmidta бабуська
2851,Как же лажает ЛаБри в Chaos in Motion. Это ж ужас просто :(,negative,лажаета лабри chaos motion ужас просто
17840,"@bouvens ладно, тогда спасай от смерти, как хочешь)) и от какой хочешь))",positive,bouvens ладный спасать смерть хотеть хотеть


In [ ]:
df.to_csv('twit_lemmatized.tsv', sep='\t', encoding='utf-8')

In [ ]:
df = pd.read_csv('twit_lemmatized.tsv', sep='\t', index_col=0, encoding='utf-8')
df.lemmas = df.lemmas.values.astype('U')
df.sample(5)

,text,label,lemmas
33177,@ndjz1997 нам только вчера сказали на него деньги сдавать!(,negative,ndjz вчера сказать деньга сдавать
94450,1с: зарплата и управление персоналомпрограмма курса ( 40 ак,negative,зарплата управление персоналомпрограмма курс
45299,"@uh_uh_uh_ сук,откуда они в бутылочке-то взялись (",negative,uh_uh_uh_ сука откуда бутылочка взяться
102322,"@saida_muser сижу на истории, уже 2 урок :(",negative,saida_muser сидеть история урок
87931,"Блин, нужно ехать в жопу мира в Металлург((",negative,блин нужно ехать жопа металлург


Разбиваем на train и test (по умолчанию 75% на 25%):

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df.lemmas, df.label)

## Мешок слов (Bag of Words, BoW)


[sklearn](https://scikit-learn.org/stable/) - библиотека для машинного обучения

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

Что такое n-граммы:

In [ ]:
from nltk import ngrams

In [ ]:
sent = 'Каждый новый день - лишь один из дней под луной, под лампой, под потолком.'.split()
list(ngrams(sent, 1)) # униграммы

[('Каждый',),
 ('новый',),
 ('день',),
 ('-',),
 ('лишь',),
 ('один',),
 ('из',),
 ('дней',),
 ('под',),
 ('луной,',),
 ('под',),
 ('лампой,',),
 ('под',),
 ('потолком.',)]

In [ ]:
list(ngrams(sent, 2)) # биграммы

[('Каждый', 'новый'),
 ('новый', 'день'),
 ('день', '-'),
 ('-', 'лишь'),
 ('лишь', 'один'),
 ('один', 'из'),
 ('из', 'дней'),
 ('дней', 'под'),
 ('под', 'луной,'),
 ('луной,', 'под'),
 ('под', 'лампой,'),
 ('лампой,', 'под'),
 ('под', 'потолком.')]

In [ ]:
list(ngrams(sent, 5)) # пентаграммы?

[('Каждый', 'новый', 'день', '-', 'лишь'),
 ('новый', 'день', '-', 'лишь', 'один'),
 ('день', '-', 'лишь', 'один', 'из'),
 ('-', 'лишь', 'один', 'из', 'дней'),
 ('лишь', 'один', 'из', 'дней', 'под'),
 ('один', 'из', 'дней', 'под', 'луной,'),
 ('из', 'дней', 'под', 'луной,', 'под'),
 ('дней', 'под', 'луной,', 'под', 'лампой,'),
 ('под', 'луной,', 'под', 'лампой,', 'под'),
 ('луной,', 'под', 'лампой,', 'под', 'потолком.')]

Итак, мы хотим преобразовать наши обработанные данные в вектора с помощью мешка слов. Мешок слов можно строить как для отдельных слов (лемм в нашем случае), так и для n-грамм, и это может улучшать качество.

Объект `CountVectorizer` делает простую вещь:
* строит для каждого документа (каждой пришедшей ему строки) вектор размерности `n`, где `n` -- количество слов или n-грам во всём корпусе
* заполняет каждый i-тый элемент количеством вхождений слова в данный документ

In [ ]:
vec = CountVectorizer(ngram_range=(1, 1)) # строим BoW для слов
bow = vec.fit_transform(x_train)

ngram_range отвечает за то, какие n-граммы мы используем в качестве признаков:<br/>
ngram_range=(1, 1) -- униграммы<br/>
ngram_range=(3, 3) -- триграммы<br/>
ngram_range=(1, 3) -- униграммы, биграммы и триграммы.

В vec.vocabulary_ лежит словарь: соответствие слов и их индексов в словаре:

In [ ]:
list(vec.vocabulary_.items())[:10]

[('случайно', 151249),
 ('пойти', 139549),
 ('учёба', 160541),
 ('большой', 97721),
 ('повториться', 138378),
 ('halcyon_o_o', 29556),
 ('потрясать', 141341),
 ('фильм', 161210),
 ('актёрский', 92584),
 ('состав', 152463)]

In [ ]:
bow.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
len(bow.toarray()[1])

168766

Теперь у нас есть вектора, на которых можно обучать модели!

In [ ]:
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)

LogisticRegression(max_iter=500, random_state=42)

Посмотрим на качество классификации на тестовой выборке. Для этого выведем classification_report из модуля [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics)

В качестве целевой метрики качества будем рассматривать macro average f1-score.

In [ ]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.75      0.73      0.74     28453
    positive       0.73      0.75      0.74     28256

    accuracy                           0.74     56709
   macro avg       0.74      0.74      0.74     56709
weighted avg       0.74      0.74      0.74     56709



Попробуем сделать то же самое для триграмм:

In [ ]:
vec = CountVectorizer(ngram_range=(3, 3))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter = 300)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.97      0.53      0.68     51375
    positive       0.16      0.86      0.27      5334

    accuracy                           0.56     56709
   macro avg       0.57      0.69      0.48     56709
weighted avg       0.90      0.56      0.64     56709



Видим, что качество существенно хуже.

И ещё для 1-, 2-, 3-грамм

In [ ]:
# это долго
vec = CountVectorizer(ngram_range=(1, 3))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter = 300)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.76      0.74      0.75     28695
    positive       0.74      0.76      0.75     28014

    accuracy                           0.75     56709
   macro avg       0.75      0.75      0.75     56709
weighted avg       0.75      0.75      0.75     56709



## TF-IDF векторизация

`TfidfVectorizer` делает то же, что и `CountVectorizer`, но в качестве значений – tf-idf каждого слова.

У слова высокий TF-IDF, если слово часто встречается в одном документе, но в целом по корпусу встречается в небольшом количестве документов.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vec = TfidfVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter = 500)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.70      0.74      0.72     26280
    positive       0.76      0.73      0.74     30429

    accuracy                           0.73     56709
   macro avg       0.73      0.73      0.73     56709
weighted avg       0.73      0.73      0.73     56709



В этот раз получилось хуже, чем с помощью простого CountVectorizer, то есть использование tf-idf не дало улучшений в качестве.

## О важности разумного анализа

Иногда в ходе стандартного препроцессинга теряются важные признаки. Посмотрим, что будет если не убирать пунктуацию?

In [ ]:
df.sample(5)

,text,label,lemmas
6063,RT @capimixaweki: времени нет писать про жизнь в Китае... :(,negative,capimixaweki время писать жизнь китай
75449,"до меня только что дошло, что я ничего не загадала на НГ!кааак, каак так??????\nя лузер((((",negative,дойти загадать кааак каак лузер
78722,"@billiejoeskon оооу,сегодня отлично,главное что по биологии не спросили,больше всего боялась)))\nА так все супер.\nУ тебя в шк как?",positive,billiejoeskon оооу сегодня отлично главное биология спросить большой весь бояться супер
67727,"@VeraVolf Большое спасибо,Вера!)полное имя у меня Наталья,мы же в ФБ немного переписывались)за пожелание большое спасибо)))",positive,veravolf большой спасибо вера полный наталья немного переписываться пожелание большой спасибо
78065,Утром была минутка милоты :) скорее бы уже февраль!!!,positive,утром минутка милот скорее февраль


In [ ]:
df['new_lemmas'] = df.text.apply(lambda x: x.lower())
df.sample(3)

,text,label,lemmas,new_lemmas
17138,"Молодёжка закончилась:( очень надеюсь, что будет 2 сезон!&lt;3",negative,молодёжка закончиться очень надеяться сезон,"молодёжка закончилась:( очень надеюсь, что будет 2 сезон!&lt;3"
58288,"@_marsi А, ну так я ж почти угадал - будет большая зона свободной торговли)",positive,_marsi угадать больший зона свободный торговля,"@_marsi а, ну так я ж почти угадал - будет большая зона свободной торговли)"
25194,"“@Ivashenko_tweet: В нашей стране у людей принято оскорблять всё и вся. Это черта исключительно русских.” К сожалению, это так :(",negative,ivashenko_tweet страна человек принять оскорблять весь черта исключительно русский сожаление,"“@ivashenko_tweet: в нашей стране у людей принято оскорблять всё и вся. это черта исключительно русских.” к сожалению, это так :("


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df.new_lemmas, df.label)

In [ ]:
from nltk import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter = 300)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     27922
    positive       1.00      1.00      1.00     28787

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



In [ ]:
word_tokenize('Утром была минутка милоты :) скорее бы уже февраль!!!')

['Утром',
 'была',
 'минутка',
 'милоты',
 ':',
 ')',
 'скорее',
 'бы',
 'уже',
 'февраль',
 '!',
 '!',
 '!']

Как можно видеть, если оставить пунктуацию, то все метрики равны 1.

In [ ]:
len(vec.vocabulary_), len(clf.coef_[0])

(260228, 260228)

In [ ]:
# это то, какой порядковый номер у какого токена (признака)
list(vec.vocabulary_.items())[:10]

[('``', 31302),
 ('@', 29465),
 ('samantadarko', 85334),
 (':', 29385),
 ('ретвит', 215941),
 (',', 96),
 ('если', 137422),
 ('ты', 239245),
 ('проебал', 208086),
 ('загадать', 140395)]

In [ ]:
# упорядочили токены
sorted_toks = sorted(vec.vocabulary_.items(), key=lambda item: item[1])
sorted_toks[:10]

[('!', 0),
 ('#', 1),
 ('$', 2),
 ('%', 3),
 ('&', 4),
 ("'", 5),
 ("''", 6),
 ('(', 7),
 (')', 8),
 ('*', 9)]

In [ ]:
# это то, какие коэффициенты у признаков
# это про важность наличия признака для класса 1
clf.coef_[0]

array([ 1.01005272e-01,  3.73256477e-01, -9.21144732e-02, ...,
       -5.66518225e-05,  5.75487329e-03,  8.91926315e-03])

In [ ]:
importances = list(zip(sorted_toks, clf.coef_[0]))
importances[:10]

[(('!', 0), 0.1010052718246332),
 (('#', 1), 0.37325647707424414),
 (('$', 2), -0.09211447321755932),
 (('%', 3), 1.3891958304949708),
 (('&', 4), -0.25110743429354676),
 (("'", 5), -0.019331142846372704),
 (("''", 6), 0.19958466426184218),
 (('(', 7), -59.705312218075875),
 ((')', 8), 58.44309527710594),
 (('*', 9), 7.349875473879972)]

In [ ]:
# самые важные токены для класса 1
sorted_importances = sorted(importances, key = lambda x: -x[1])
sorted_importances[:20]

[((')', 8), 58.44309527710594),
 (('d', 43546), 26.91344786674441),
 (('dd', 44979), 10.625051537552853),
 (('^_^', 29481), 9.132530760293815),
 (('ddd', 44988), 8.03046578846086),
 (('-d', 245), 7.444101317870151),
 (('*', 9), 7.349875473879972),
 ((':', 29385), 6.103079766353063),
 (('dddd', 44989), 4.890694361315784),
 (('ddddd', 44991), 3.1638561716428617),
 (('люблю', 163320), 1.9067458364421082),
 (('dddddd', 44992), 1.777088868542044),
 (('х', 247371), 1.7094699789990497),
 (('=^_^=', 29437), 1.597157765599104),
 (('спасибо', 228082), 1.4808592616392815),
 (('%', 3), 1.3891958304949708),
 (('okirilyuk', 77921), 1.348509174999615),
 (('приятно', 207120), 1.2998294262952363),
 (('ахахах', 106326), 1.2979616732700872),
 (('dddddddd', 44994), 1.2623118168556864)]

In [ ]:
# самые важные токены для класса 0
sorted_importances = sorted(importances, key = lambda x: x[1])
sorted_importances[:20]

[(('(', 7), -59.705312218075875),
 (('|', 101408), -11.136903421619358),
 (('о_о', 180270), -10.832795333500412),
 (('-/', 130), -8.727387426025608),
 (('o_o', 77554), -8.42072340741087),
 (('cio_optimal', 42460), -5.452184278085164),
 (('to_over_kill', 92787), -5.303690459483595),
 (('do_or_die_xxx', 46765), -4.469405729573747),
 (('prisonero_o', 81780), -4.085958502396247),
 (('rt', 84443), -4.075767291412801),
 (('reno_oppa', 83409), -3.878522660579768),
 (('horanso_on', 55138), -3.633290128546062),
 (('kota_oo_oo', 62877), -3.2575586916189607),
 (('_do_or_die__', 30114), -2.7983028137047263),
 (('radio_of_moon', 82716), -2.766423693858961),
 (('boo_ohoo', 40331), -2.736179496751567),
 (('o_obnulyay', 77559), -2.55022407266154),
 (('снаступающимтвиттерский', 225405), -2.5404461659868924),
 (('lponomarenko_o', 67743), -2.311360891449097),
 (('july_to_october', 59175), -2.1480612447580416)]

In [ ]:
vec.vocabulary_[')']

8

In [ ]:
clf.coef_[0][8]

58.44309527710594

Посмотрим, как один из наиболее значительных токенов справится с классификацией безо всякого машинного обучения:

In [ ]:
cool_token = ')'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      0.85      0.92     32894
    positive       0.83      1.00      0.91     23815

    accuracy                           0.91     56709
   macro avg       0.91      0.93      0.91     56709
weighted avg       0.93      0.91      0.91     56709



Можно видеть, что это уже позволяет достаточно хорошо классифицировать тексты.

## Символьные n-граммы

Теперь в качестве признаком используем, например, униграммы символов:

In [ ]:
vec = CountVectorizer(analyzer='char', ngram_range=(1, 1))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

    negative       0.99      1.00      1.00     27876
    positive       1.00      0.99      1.00     28833

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



Таким образом, становится понятно, почему на этих данных качество классификации 1. Так или иначе, на символах классифицировать тоже можно.

Ещё одна замечательная особенность символьных признаков: токенизация и лемматизация не нужна, можно использовать такой подход для языков, у которых нет готовых анализаторов.

## Итого

* познакомились с задачей бинарной классификации текстов.

* научились строить простые признаки на основе метода "мешка слов" с помощью библиотеки sklearn: CountVectorizer и TfidfVectorizer.

* использовали для классификации линейную модель логистической регрессии.

* поняли, что многое зависит от подхода к предобработки текста и от признаков, которые используются в модели.

* увидели, что в некоторых задачах важно использование каждого символа из текста, в том числе пунктуации.